In [1]:
# relative MTADelayPredict Project
import sys
import os
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath(os.path.join('DataExploration.ipynb')))))
from MTADelayPredict.utils import gtfs_loader, stop_info
from MTADelayPredict.subway_line import SubwayLine, N_STOP_LIST
from MTADelayPredict.stop import Stop
from importlib import reload
from MTADelayPredict.plotting import alerts

In [2]:
import re

subway_line = SubwayLine(N_STOP_LIST)
obs_stop = subway_line.stop('R16N')

def stop_processor(name):
    name = str(name)
    if str.lower(name) == 'nan':
        return ''
    
    # Strip parenthesis borough delimeters, we know what line we're on
    stop_ids = stop_info.name2stop_ids(name, N_STOP_LIST)
    
    # If there were issues parsing, check a few things
    if len(stop_ids) == 0:
        # Is this a range? 
        range_m = re.match(r'^(.+),(.+)', name)
        if range_m:
            stop_1 = subway_line.stop(stop_info.name2stop_ids(range_m.groups()[0], N_STOP_LIST).iloc[0])
            stop_2 = subway_line.stop(stop_info.name2stop_ids(range_m.groups()[1], N_STOP_LIST).iloc[0])
            
            if abs(stop_1.stop_idx - obs_stop.stop_idx) < abs(stop_2.stop_idx - obs_stop.stop_idx):
                return stop_1.stop_id
            else:
                return stop_2.stop_id
            
        # Still unparsable
        return name
        
    return stop_ids.iloc[0]

In [3]:
import pandas as pd
import numpy as np
import os 

alert_dir = '../data/raw/alerts'
annotated_alert_df = pd.read_csv(os.path.abspath(os.path.join(alert_dir, 'nqrw_alerts.csv')))

n_alert_df = annotated_alert_df[annotated_alert_df['Direction'] == 'Northbound']
n_alert_df['IssueStopID'] = n_alert_df['IssueStop'].map(stop_processor)
n_alert_df = n_alert_df.reset_index()
n_alert_df = n_alert_df.drop(columns='index')

/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [4]:
import plotly.graph_objects as go
import plotly.express as px

weekday_dict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

def create_mta_wait(feature_df, alert_stop, observing_stop, date, start_time, end_time, stop_id, feature, transform):

    traces = []
#     next_scheduled_df = data['next_scheduled_arrival']
#     schedule_df = data['schedule_df']
    
    ##### Set up MTA predicted arrival time ############
#     OBSERVED_STOP = 'R16N'
#     mta_predicted = next_scheduled_df[OBSERVED_STOP].map(lambda x: pd.to_datetime(x, unit='m', utc=True).tz_convert('US/Eastern')).to_frame()
#     mta_predicted = mta_predicted.resample('1T').min()
#     mta_predicted['time'] = mta_predicted.index.shift(-1)

#     mta_predicted['wait'] = (mta_predicted[OBSERVED_STOP] - mta_predicted['time'])
#     mta_predicted['wait'] = mta_predicted['wait'].map(lambda x: x.delta// 60000000000 if isinstance(x, pd.Timedelta) else x)
#     mta_predicted.dropna(how='any', inplace=True)
    
#     subway_line = SubwayLine(N_STOP_LIST)
    
#     t = go.Scatter(x=mta_predicted['time'],
#                    y=mta_predicted['wait'],
#                    mode='lines+markers',
#                    name='MTA Predicted Wait',
#                    x0=schedule_df['index'].iloc[0]
#                   )
#     traces.append(t)
    
    ###### Plot different features ######

    feature_df.index = feature_df.index.map(lambda x: x.tz_localize('UTC').tz_convert('US/Eastern'))
    
#    print(feature_df)
#    print((feature, transform, stop_id))
    
    t = go.Scatter(x=feature_df.index,
                   y=feature_df.loc[:, (feature, transform, stop_id)],
                   mode='lines+markers',
                   name='feature:{} transform: {} stop: {}'.format(feature, transform, stop_info.stop_id2name(observing_stop)),
                   x0=start_time
                  )
    traces.append(t)    

#     t = go.Scatter(x=min_since_train.index,
#                    y=min_since_train,
#                    mode='lines+markers',
#                    name='{} min_since_train'.format(stop_info.stop_id2name(observing_stop)),
#                    x0=schedule_df['index'].iloc[0]
#                   )
#     traces.append(t)
    
    shapes = list()

    # Alert time
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': date,
                   'x1': date,
                   'line_color': 'red'
                  })

    layout = go.Layout(
                title= ('Feature {} Transform {} Stop {}'.format(feature, transform, stop_info.stop_id2name(stop_id))),
                titlefont=dict(
                family='Courier New, monospace',
                size=15,
                color='#7f7f7f'
                ),
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                width=1500,
                height=500,
                margin={'l':200},

                xaxis=dict(
                    tickmode = 'array',
#                    tickvals = schedule_df['index'],
#                    ticktext = schedule_df['index'],
                    title='Time'
                ),
                yaxis=dict(
                    tickmode = 'array',
                    title='Minutes',
                ),
                shapes=shapes
    )
    return {'data':traces, 'layout':layout}

In [5]:
import plotly.graph_objects as go
import plotly.express as px

weekday_dict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

def create_schedules(schedule_df, alert_stop, observing_stop, date, start_time, end_time):
    traces = []
    
    subway_line = SubwayLine(N_STOP_LIST)

    schedule_df['index'] = schedule_df['index'].map(lambda x: pd.to_datetime(x, unit='ms', utc=True).tz_convert('US/Eastern'))
    schedule_df[schedule_df['index'] > start_time]
    
 #   print(schedule_df)
    
    for col in schedule_df.columns[2:]:
        plot_df = schedule_df[['index', col]].dropna()
        if (plot_df.shape[0] == 0 or plot_df['index'].iloc[-1] < start_time) or (plot_df['index'].iloc[0] > end_time):
            continue

        t = go.Scatter(
                    x= plot_df['index'],
                    y = plot_df[col],
                    mode='lines+markers',
                    name=col,
                    x0=schedule_df['index'].iloc[0],
                    showlegend=False
            )

        traces.append(t)
    shapes = list()

    # Alert time
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': date,
                   'x1': date,
                   'y0': 0,
                   'y1': schedule_df.iloc[:, 2:].max().max(),
                   'line_color': 'red'
                  })

    # Alert stop
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': schedule_df['index'].min(),
                   'x1': schedule_df['index'].max(),
                   'y0': subway_line.stop_idx(alert_stop),
                   'y1': subway_line.stop_idx(alert_stop),
                   'line_color': 'red'
                  })

    # Observed stop
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': schedule_df['index'].min(),
                   'x1': schedule_df['index'].max(),
                   'y0': subway_line.stop_idx(observing_stop),
                   'y1': subway_line.stop_idx(observing_stop),
                   'line_color': 'green'
                  })

    layout = go.Layout(
                title= ('Alert @ {} {}'.format(date, weekday_dict[date.dayofweek])),
                titlefont=dict(
                family='Courier New, monospace',
                size=15,
                color='#7f7f7f'
                ),
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                width=1600,
                height=800,
                margin={'l':200},

                xaxis=dict(
                    tickmode = 'array',
                    tickvals = schedule_df['index'],
                    ticktext = schedule_df['index'],
                ),
                yaxis=dict(
                    tickmode = 'array',
                    tickvals = [i for i,_ in enumerate(N_STOP_LIST)],
                    ticktext = [stop_info.stop_id2name(s) for s in N_STOP_LIST],
                    title='Stop',
                ),
                shapes=shapes
    )
    return {'data':traces, 'layout':layout}


In [6]:
def build_features(df):
    features = []
    features.append(df)
    features.append(df.rolling('5T').max())
    features.append(df.rolling('15T').max())
    features.append(df.rolling('30T').max())
    features.append(df.rolling('45T').max())
    features.append(df.rolling('60T').max())
    features.append(df.rolling('90T').max())
    return pd.concat(features, axis=1, keys=['minutes', 'max5', 'max15', 'max30', 'max45', 'max60', 'max90'], names=['transform'])

In [ ]:
from datetime import datetime as dt
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import dash_table
import re
from dash.exceptions import PreventUpdate
from plotly.subplots import make_subplots
from flask import Flask
from flask_caching import Cache
import redis

external_stylesheets = [    # Dash CSS
    'https://codepen.io/chriddyp/pen/bWLwgP.css',
    # Loading screen CSS
    'https://codepen.io/chriddyp/pen/brPBPO.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
CACHE_CONFIG = {
    # try 'filesystem' if you don't want to setup redis
    'CACHE_TYPE': 'redis',
    'CACHE_REDIS_URL': os.environ.get('REDIS_URL', 'redis://localhost:6379')
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

transforms = ['minutes', 'max5', 'max15', 'max30', 'max45', 'max60', 'max90']
features = ['min_in_station', 'min_since_train']
stop_ids = N_STOP_LIST

transform_dropdown = [{'label':x, 'value':x} for x in transforms ]
feature_dropdown = [{'label':x, 'value':x} for x in features ]
stop_dropdown = [{'label':stop_info.stop_id2name(x), 'value':x} for x in stop_ids]


dropdown_values = []
for idx,row in n_alert_df.iterrows():
    dropdown_values.append({'label':"{} {}".format(row.Date, row.Subject), 'value': idx})

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(1995, 8, 5),
        max_date_allowed=dt(2017, 9, 19),
        initial_visible_month=dt(2017, 8, 5),
        date=str(dt(2017, 8, 25, 23, 59, 59))
    ),
#    html.Button('Previous', id='prev-button', n_clicks=0),
#    html.Button('Next', id='next-button', n_clicks=0),
    dcc.Dropdown(
        id='alert-dropdown',
        options=dropdown_values,
        value=1
    ),
    html.Div(id='current-alert'),
    html.Div(id='alert-subject'),
    html.Div(id='alert-location'),
    html.Div(id='alert-message'),
    dcc.Graph(id='alert-graph'),
    dcc.Graph(id='features-graph'),
    dcc.Dropdown(
        id='transform-dropdown',
        options=transform_dropdown,
        value=transforms[0]
    ),
    dcc.Dropdown(
        id='feature-dropdown',
        options=feature_dropdown,
        value=features[0]
    ),
    dcc.Dropdown(
        id='stop-dropdown',
        options=stop_dropdown,
        value=stop_ids[0]
    ),
    dcc.Graph(id='wait-graph'),
    # Hidden div to keep track of what idx we're on
    html.Div(id='schedule-df', style={'display': 'none'}),
    html.Div(id='features-df', style={'display': 'none'}),
])

@app.callback(Output('wait-graph', 'figure'),
             [Input('alert-dropdown', 'value')])
def update_wait(idx):
    schedule_df, feature_df = update_data(idx)
    
    WAIT_THRESHOLD = 15
    OBSERVED_STOP = 'R16N'
    traces = []

    feature_df.index = feature_df.index.map(lambda x: x.tz_localize('UTC').tz_convert('US/Eastern'))

    t = go.Scatter(x=feature_df.index,
                   y=feature_df.loc[:, ('min_since_train', 'minutes', OBSERVED_STOP)],
                   mode='lines+markers',
                   name='wait_time @ {}'.format(stop_info.stop_id2name(OBSERVED_STOP)),
                  )
    traces.append(t)    
    

    predicted_value = feature_df.loc[:, ('min_since_train', 'minutes', OBSERVED_STOP)].fillna(method='ffill')
    predicted_value = predicted_value.rolling('15T').max().shift(-15)
    predicted_value[predicted_value < WAIT_THRESHOLD] = 0
    predicted_value[predicted_value >= WAIT_THRESHOLD] = 1
    
#     t = go.Scatter(x=predicted_value.index,
#                    y=predicted_value,
#                    mode='lines+markers',
#                    name='is_delay @ {}'.format(stop_info.stop_id2name(OBSERVED_STOP)),
#                   )
    t = go.Bar(x=predicted_value.index,
                   y=predicted_value,
                   name='is_delay @ {}'.format(stop_info.stop_id2name(OBSERVED_STOP)),
                   marker_line_width=1.5, opacity=0.6,
                  )
    traces.append(t) 
    
    shapes = list()

    # Alert time
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': feature_df.index[0],
                   'x1': feature_df.index[-1],
                   'y0': WAIT_THRESHOLD,
                   'y1': WAIT_THRESHOLD,
                   'line_color': 'red'
                  })
    
    
    layout = go.Layout(
                title= ('Train Wait @ {}'.format(stop_info.stop_id2name(OBSERVED_STOP))),
                titlefont=dict(
                family='Courier New, monospace',
                size=15,
                color='#7f7f7f'
                ),
                paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                width=1600,
                height=800,
                margin={'l':200},

                xaxis=dict(
                    tickmode = 'array',
                    tickvals = schedule_df['index'],
                    ticktext = schedule_df['index'],
                ),
                yaxis=dict(
                    tickmode = 'array',
                    title='minutes',
                ),
                shapes=shapes
    )    
    
    wait_fig = go.Figure(data=traces, layout=layout)
    return wait_fig
    
@app.callback(Output('features-graph', 'figure'),
             [Input('alert-dropdown', 'value'), Input('transform-dropdown', 'value'), Input('feature-dropdown', 'value'), Input('stop-dropdown', 'value')])
def update_features(idx, transform, feature, stop_id):
    schedule_df, feature_df = update_data(idx)
    
    start_window = 120
    end_window = 120
    OBSERVED_STOP = 'R16N'
    row = n_alert_df.iloc[int(idx)]

#    print("At index {} [{}]".format(idx, row.IssueStopID))
#    print(row)
    alert_time = pd.Timestamp(row.Date).tz_localize("US/Eastern")
    alert_stop = row.IssueStopID
    if alert_stop == '':
        alert_stop = 'R16N'

    STOP_FILTER = '^.*N$'
    ROUTE_FILTER = 'N'
    data_dir = '../data/raw/status'

    start_time = alert_time - pd.Timedelta(start_window, unit='m')
    end_time = alert_time + pd.Timedelta(end_window, unit='m')
    
    schedule_dict =  create_schedules(schedule_df, alert_stop, OBSERVED_STOP, alert_time, start_time, end_time)
    
    
    mta_wait_dict = create_mta_wait(feature_df, alert_stop, OBSERVED_STOP, alert_time, start_time, end_time, feature=feature, transform=transform, stop_id=stop_id)
    
    feature_fig = go.Figure(mta_wait_dict)
    feature_fig.update_xaxes(schedule_dict['layout']['xaxis'])
    return feature_fig

@app.callback(
    [Output('alert-subject', 'children'), Output('alert-location', 'children'), Output('alert-message', 'children')],
[Input('alert-dropdown', 'value')])
def display_alert(alert_idx):
    row = n_alert_df.iloc[int(alert_idx)]
    return 'Subject: '+row['Subject'], 'IssueStopID: '+ stop_info.stop_id2name(row['IssueStopID']), 'Message: '+row['Message']
    
@app.callback(
    Output('current-alert', 'children'),
    [Input('alert-dropdown', 'value')])
def update_output(alert_idx):
    string_prefix = 'You have selected: '    
    row = n_alert_df.iloc[int(alert_idx)]

    date = row.Date
    if date is not None:
        date = pd.Timestamp(str(date))
        return string_prefix + str(date)
    

def update_data(idx):
    @cache.memoize()
    def query_and_serialize_data(idx):
        start_window = 120
        end_window = 120
        OBSERVED_STOP = 'R16N'
        row = n_alert_df.iloc[int(idx)]

    #    print("At index {} [{}]".format(idx, row.IssueStopID))
    #    print(row)
        alert_time = pd.Timestamp(row.Date).tz_localize("US/Eastern")
        alert_stop = row.IssueStopID
        if alert_stop == '':
            alert_stop = 'R16N'

        STOP_FILTER = '^.*N$'
        ROUTE_FILTER = 'N'
        data_dir = '../data/raw/status'

        start_time = alert_time - pd.Timedelta(start_window, unit='m')
        end_time = alert_time + pd.Timedelta(end_window, unit='m')

        # Fetch schedule data and plot
        data = alerts.load_range_schedule(start_time, end_time, STOP_FILTER, ROUTE_FILTER, data_dir)

        loader = data['loader']
        schedule_dfs = []
        for stop, train in zip(loader.stop_dict.items(), loader.train_dict.items()):
            schedule_dfs.append(pd.DataFrame(stop[1], index=train[1], columns=[stop[0]]))
        line = SubwayLine(N_STOP_LIST)
        df = pd.concat(schedule_dfs)
        df = df.sort_index()[start_time:end_time]


        df = df.stack().reset_index()
        df.columns = ['time', 'train_id', 'stop_id']
        df = df.pivot_table(index='time', columns='stop_id', values='train_id', aggfunc=np.max)
        df.sort_index(inplace=True)
        df = df.resample('1T').last()

        min_in_station = df.copy()
        min_in_station[~df.isna()] = 1.0
        min_in_station = min_in_station.astype(float)

        train_gaps = df.copy().shift(1)
        train_gaps[~df.isna()] = 0.0
        train_gaps.fillna(method='ffill', inplace=True)
        train_gaps.replace(0.0, np.nan, inplace=True)
        min_since_train = train_gaps.copy()

        min_since_train[~train_gaps.isna()] = 1.0
        min_since_train = min_since_train.astype(float)

        for observing_stop in df.columns:
            min_since_train[observing_stop] = min_since_train[observing_stop].groupby(train_gaps[observing_stop]).cumsum()
            min_in_station[observing_stop] = min_in_station[observing_stop].groupby(df[observing_stop]).cumsum()

        min_in_station = build_features(min_in_station)
        min_since_train = build_features(min_since_train)
        features_df = pd.concat([min_in_station, min_since_train], axis=1, keys=['min_in_station', 'min_since_train'], names=['feature'])
        
        return [data['schedule_df'].to_json(), features_df.to_json()]
    schedule_json, feature_json = query_and_serialize_data(idx)
    feature_df = pd.read_json(feature_json)
    
    def str2tuple(s):
        import re
        foo = re.sub(r'[\"\'\(\) ]', '', s)
        return tuple(foo.split(','))
        
    feature_df.columns = pd.MultiIndex.from_tuples(feature_df.columns.map(str2tuple))
#    print(feature_df.columns)
#    feature_df.columns = pd.MultiIndex.from_tuples(feature_df.columns)
    return [pd.read_json(schedule_json), feature_df]

    
    
@app.callback(
    Output('alert-graph', 'figure'),
[Input('alert-dropdown', 'value')])
def update_plot(idx):
    start_window = 120
    end_window = 120
    OBSERVED_STOP = 'R16N'
    row = n_alert_df.iloc[int(idx)]

#    print("At index {} [{}]".format(idx, row.IssueStopID))
#    print(row)
    alert_time = pd.Timestamp(row.Date).tz_localize("US/Eastern")
    alert_stop = row.IssueStopID
    if alert_stop == '':
        alert_stop = 'R16N'
    
    STOP_FILTER = '^.*N$'
    ROUTE_FILTER = 'N'
    data_dir = '../data/raw/status'
    
    start_time = alert_time - pd.Timedelta(start_window, unit='m')
    end_time = alert_time + pd.Timedelta(end_window, unit='m')

    schedule_df, feature_df = update_data(idx)
    
    print("Loading {} - {}".format(start_time, end_time))
    schedule_dict =  create_schedules(schedule_df, alert_stop, OBSERVED_STOP, alert_time, start_time, end_time)
    
    schedule_fig = go.Figure(schedule_dict)

    return schedule_fig
    
#    fig = go.Figure()
#    fig.show()
#    return schedule_dict, mta_wait_dict
    
app.run_server(host='0.0.0.0', port='8050', debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter

Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Running on http://0.0.0.0:8050/
Debugger PIN: 376-798-318
Debugger PIN: 376-798-318
Debugger PIN: 376-798-318
Debugger PIN: 376-798-318
Debugger PIN: 376-798-318
Debugger PIN: 376-798-318
Debugger PIN

Exception possibly due to cache backend.
Traceback (most recent call last):
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 795, in decorated_function
    f, *args, **kwargs
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 592, in make_cache_key
    f, args=args, timeout=_timeout, forced_update=forced_update
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 543, in _memoize_version
    version_data_list = list(self.cache.get_many(*fetch_keys))
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 239, in cache
    return app.extensions["cache"][self]
KeyError: 'cache'
Exception possibly due to cache backend.
Traceback (most recent call last):
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 795, in decorated_function
    f, *args, **kwargs
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 592, in make_c

25552526
25552646
2555252625552526
25552646

25552646


  0%|                                     |entries: ------decode_errors: ------Exception possibly due to cache backend.
Traceback (most recent call last):
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 795, in decorated_function
    f, *args, **kwargs
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 592, in make_cache_key
    f, args=args, timeout=_timeout, forced_update=forced_update
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 543, in _memoize_version
    version_data_list = list(self.cache.get_many(*fetch_keys))
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init__.py", line 239, in cache
    return app.extensions["cache"][self]
KeyError: 'cache'
  6%|##                                   |entries: ------decode_errors: ------Exception possibly due to cache backend.
Traceback (most recent call last):
  File "/miniconda3/lib/python3.6/site-packages/flask_caching/__init

25552526
25552646
25552526
25552646
25552526
25552646


...

New stops:
set()


100%|#####################################|entries: ------decode_errors: ------


New stops:
set()


100%|#####################################|entries: ------decode_errors: ------


New stops:
set()
Loading 2018-08-27 06:53:00-04:00 - 2018-08-27 10:53:00-04:00
